## 11 Hog + SVM 小狮子识别
- 1.2. Hog特征+SVM分类器简单介绍
- 3. 小狮子识别例子

#### 1. Hog特征

image>win>block>cell，cell是最小的绿色的.<br>
- **win：**必须包含了一个obj所有的描述信息，根据这个窗体计算出来的信息才能唯一表征这个obj.<br>
`win_size`：官方一般推荐：64\*128，自己定义为50\*100，20\*50都OK
- **block：**`win_size`是`block_size`整数倍，在win里面滑动，有block_step<br>
block_count in win = $((winW-blockW)/block\_stepW+1)*((winH-blockH)/block\_stepH+1)$
- **cell：**填充block，一个block中所含数目为`cell_count`个。一个cell包含9个bin，完整的360度。
- **bin：**与方向有关。0~360/40 = 9个单元，每个单元称为一个bin<br>
每个像素都需要计算梯度（包含大小f、方向angle）

<img src='pic_Markdown/hog.png' width="40%">


- **特征维度：**haar特征得到是一个值，hog是一个向量（维度）。<br>
维度 = block_count * cell_count * bin_count(9)
- **梯度计算、特征模板：**用特征模板计算梯度，每个像素都会计算一个梯度。<br>
水平方向：[1 0 -1] 竖直方向：[[1] [0] [-1]]<br>
$i.e:$<br>
a = p1\*1 +p2\*0 + p3\*(-1) = 相邻像素之差<br>
b = 上下像素之差<br>
$f = \sqrt{a^2+b^2}，angle=arctan(a/b)$ 
- **bin计算、投影、梯度：**
    - 判断(i,j)属于哪个bin？bin1范围——(0,20)or(180,200),这两个范围相差180度。
    - 投影f：根据angle分配到不同的bin。例如angle=10，正好落在(0,20)中间，不分配。例如angle=25，则按比例分到bin1，bin2；or不分配，直接落到某个bin里面也是可以的。

- **整体hog计算：**遍历所有像素，计算bin，然后sumbin0()，p.s: 位于bin边缘的一些点(i,j)的幅值f可能投影到相邻bin，所以计算sumbin0()时，要考虑相邻像素在(i,j)上的投影
- **cell复用：**由于上面提到那一点，有另外一种cell的分类方式，cellx0、cellx2、cellx4，分别代表：(i,j)对当前bin、两个bin、四个bin有影响

#### 2. SVM分类器
kernel : line <br>
数据：<br>
训练: SVM_creat，train，predict<br>
hog训练完成后，是3780维；svm line训练出[3780]，hog\*svm = 值

---



In [1]:
# —— SVM简单例子 根据身高判断性别 ——

import numpy as np
import cv2
import matplotlib.pyplot as plt
# 女生 [身高,体重]
rand1= np.array([[155,48],[159,50],[164,53],[168,56],[172,60]])
rand2= np.array([[152,53],[156,55],[160,56],[172,64],[176,65]])
# print(rand1)

# 2 label
label = np.array([[0],[0],[0],[0],[0],[1],[1],[1],[1],[1]])

# 3 data 处理  
data=np.vstack((rand1,rand2)) # rand1,2 合并为元祖
# print(data)
# data = np.array(data,dtype='float32') 
data=np.array(data,np.float32)
# print(data)

# svm 所有数据都要有标签
# [155,48] -- 0  女  [152,53] --1 男
 
# 4 训练
svm=cv2.ml.SVM_create() # ml 机器学习模块,创建SVM
# 属性设置
svm.setType(cv2.ml.SVM_C_SVC)  # SVM类型
svm.setKernel(cv2.ml.SVM_LINEAR) # 线性分类器
svm.setC(0.01)
# 训练
result=svm.train(data,cv2.ml.ROW_SAMPLE,label)
# 预测 看训练效果
pt_data=np.vstack([[167,55],[162,57]]) # 0 女 1 男
pt_data=np.array(pt_data,dtype='float32')
print(pt_data)
(par1,par2)=svm.predict(pt_data)
print(par2)



[[ 167.   55.]
 [ 162.   57.]]
[[ 0.]
 [ 1.]]


In [ ]:
# Hog特征
# image win block cell 分析四者的关系和size
# if win= 64*128 then,block = 16*16,cell=8*8(4个cell)
# block_step=8*8
# block_count=((64-16)/8+1)*((128-16)/8+1)=105个block
# 维度 = 105*4*9=3780 


In [4]:
# 训练
# 1 参数 2 hog 3 svm 4 computer hog 5 label
# 6 train 7 predict 8 draw

import numpy as np
import cv2
import matplotlib.pyplot as plt
# 1 参数定义
PosNum = 820
NegNum = 1931
winSize = (64, 128)
blockSize = (16, 16)  # win有105个block
blockStride = (8, 8)
cellSize = (8, 8)
nBin = 9  # 最终bin个数 = 3780,也是特征维度

# 2 创建hog
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nBin)
# 3 创建svm
svm = cv2.ml.SVM_create()
# 4 计算hog
# 初始化一些数组来保存hog参数
# featureNum也可以通过HOGDescriptor的维度信息来获得
featureNum = int(
    ((128 - 16) / 8 + 1) * ((64 - 16) / 8 + 1) * 4 * 9)  # 3780 特征维度
featureArray = np.zeros(((PosNum + NegNum), featureNum), np.float32)
labelArray = np.zeros(((PosNum + NegNum), 1), np.int32)
# svm监督学习 需要有样本和标签 svm学习的是hog特征
# featureArray才是它真正的样本 而不是PicData

# 加载数据
for i in range(0, PosNum):
    fileName = 'lionData/pos/' + str(i + 1) + '.jpg'
    img = cv2.imread(fileName)  # BGR
    hist = hog.compute(img, (8, 8))  # 3780维
    # featureArray[i,:]=hist[:]
    for j in range(0, featureNum):
        featureArray[i, j] = hist[j]
    # 正样本 label=1
    labelArray[i, 0] = 1
for i in range(0, NegNum):
    fileName = 'lionData/neg/' + str(i + 1) + '.jpg'
    img = cv2.imread(fileName)  # BGR
    hist = hog.compute(img, (8, 8))  # 3780维
    # featureArray[i+PosNum,:]=hist[:]
    for j in range(0, featureNum):
        featureArray[i + PosNum, j] = hist[j]
    # 负样本 label = -1
    labelArray[i + PosNum, 0] = -1

# SVM属性设置
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setC(0.01)

# 6 train
ret = svm.train(featureArray, cv2.ml.ROW_SAMPLE, labelArray)
# 7 检测 核心：myhog
# 理解：create Hog <- myDetect <- array <- resultArray+rho
#      rho <- svm
#      resultArray <- supportVArray
#      myhog -> detectMultiScale完成检测
#               没使用predetect因为前者可以返回obj的宽高信息,才能绘制出来
alpha = np.zeros((1), np.float32)
rho = svm.getDecisionFunction(0, alpha) # 用于最后阈值判决,累加
print('rho:',rho)
print('alpha:',alpha)
# alphaArray * supportVArray(支持向量机数组)
alphaArray = np.zeros((1, 1), np.float32)
supportVArray = np.zeros((1, featureNum), np.float32)
resultArray = np.zeros((1, featureNum), np.float32)
alphaArray[0, 0] = alpha
resultArray = -1 * alphaArray * supportVArray
# 构建detect
myDetect = np.zeros((3781), np.float32)
for i in range(0, 3780):
    myDetect[i] = resultArray[0, i]  # 3780个全部放入
myDetect[3780] = rho[0]  # 第3781放入rho[0]
# 构建myhog
myHog = cv2.HOGDescriptor()
myHog.setSVMDetector(myDetect) # 把myDetect属性传递进去
# 读取待检测图片
imgSrc = cv2.imread('lionData/test.jpg', 1)
# objs三维,(8,8)滑动步长 (32,32)窗体大小 1.05——缩放倍数
objs = myHog.detectMultiScale(imgSrc, 0, (8, 8), (32, 32), 1.05, 2)
# 宽高信息
x = int(objs[0][0][0])
y = int(objs[0][0][1])
w = int(objs[0][0][2])
h = int(objs[0][0][3])
print('objs:',objs)
cv2.rectangle(imgSrc, (x, y), (x + w, y + h), (255, 0, 0), 2)

cv2.imshow('img', imgSrc)
cv2.waitKey(0)
cv2.destroyAllWindows()

rho: (0.25550148582186183, array([[ 1.]]), array([[0]], dtype=int32))
alpha: [ 0.]
objs: (array([[  0,   0,  64, 128]], dtype=int32), array([[ 0.25550148]]))


In [2]:
imgSrc=cv2.imread('lionData/test2.jpg',1)
# 1.05——缩放倍数
objs=myHog.detectMultiScale(imgSrc,0,(8,8),(32,32),1.05,2)
# 宽高信息
x=int(objs[0][0][0])
y=int(objs[0][0][1])
w=int(objs[0][0][2])
h=int(objs[0][0][3])
cv2.rectangle(imgSrc,(x,y),(x+w,y+h),(255,0,0),2)

cv2.imshow('img', imgSrc)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
#训练 1 参数 2hog 3 svm 4 computer hog 5 label 6 train 7 pred 8 draw
# 检测失败？ why?? neg和pos图像，都把全部框起来了
import cv2
import numpy as np
import matplotlib.pyplot as plt
# 1 par 各个具体的参数展示
PosNum = 820
NegNum = 1931
winSize = (64, 128)
blockSize = (16, 16)  # 105
blockStride = (8, 8)  # 4 cell
cellSize = (8, 8)
nBin = 9  # 9 bin 3780
# 2 hog create hog 1 win 2 block 3 blockStride 4 cell 5 bin HOG特征的基本流程
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nBin)
# 3 svm 分类器的调用
svm = cv2.ml.SVM_create()
# 4 computer hog
featureNum = int(((128 - 16) / 8 + 1) * ((64 - 16) / 8 + 1) * 4 * 9)  # 3780
featureArray = np.zeros(((PosNum + NegNum), featureNum), np.float32)
labelArray = np.zeros(((PosNum + NegNum), 1), np.int32)
# svm 监督学习 样本 标签 svm -》image hog
for i in range(0, PosNum):
    fileName = 'lionData/pos/' + str(i + 1) + '.jpg'
    img = cv2.imread(fileName)
    hist = hog.compute(img, (8, 8))  # 3780
    for j in range(0, featureNum):
        featureArray[i, j] = hist[j]
    # featureArray hog [1,:] hog1 [2,:]hog2
    labelArray[i, 0] = 1
    # 正样本的训练数据进行学习 label 1
for i in range(0, NegNum):
    fileName = 'lionData/neg/' + str(i + 1) + '.jpg'
    img = cv2.imread(fileName)
    hist = hog.compute(img, (8, 8))  # 3780
    for j in range(0, featureNum):
        featureArray[i + PosNum, j] = hist[j]
    labelArray[i + PosNum, 0] = -1
# 负样本的训练数据进行学习 label -1
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setC(0.01)
# 6 train  对数据进行训练
ret = svm.train(featureArray, cv2.ml.ROW_SAMPLE, labelArray)
# 7 myHog ：《-myDetect
# myDetect-《resultArray  rho
# myHog-》detectMultiScale
# 7 检测  核心：create Hog -》 myDetect—》array-》
# resultArray-》resultArray = -1*alphaArray*supportVArray
# rho-》svm-〉svm.train
alpha = np.zeros((1), np.float32)
rho = svm.getDecisionFunction(0, alpha)
print(rho)
print(alpha)
alphaArray = np.zeros((1, 1), np.float32)
supportVArray = np.zeros((1, featureNum), np.float32)
resultArray = np.zeros((1, featureNum), np.float32)
alphaArray[0, 0] = alpha
resultArray = -1 * alphaArray * supportVArray
# detect
myDetect = np.zeros((3781), np.float32)
for i in range(0, 3780):
    myDetect[i] = resultArray[0, i]
myDetect[3780] = rho[0]
# rho svm （判决）
myHog = cv2.HOGDescriptor()
myHog.setSVMDetector(myDetect)
# load
imageSrc = cv2.imread('lionData/test2.jpg', 1)
# (8,8) win
objs = myHog.detectMultiScale(imageSrc, 0, (8, 8), (32, 32), 1.05, 2)
# xy wh 三维 最后一维
x = int(objs[0][0][0])
y = int(objs[0][0][1])
w = int(objs[0][0][2])
h = int(objs[0][0][3])
print('objs:',objs)
# 绘制展示
cv2.rectangle(imageSrc, (x, y), (x + w, y + h), (255, 0, 0), 2)
cv2.imshow('dst', imageSrc)
cv2.waitKey(0)
cv2.destroyAllWindows()

(0.25550148582186183, array([[ 1.]]), array([[0]], dtype=int32))
[ 0.]
objs: (array([[  0,   0,  64, 128]], dtype=int32), array([[ 0.25550148]]))
